In [3]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from chinese_calendar import is_workday
import socket
import struct
from sklearn.decomposition import PCA
import copy
import random
import time

In [4]:
#############Loading raw data#############
dfinit = pd.read_csv("data/loginmysql-db_for_exp.csv")
#################Data feature processing#######################
groups_sip_dip = dfinit.groupby(['sip','dip'])
groups_sip_dip_sum = pd.DataFrame(dfinit.groupby(['sip','dip'])['num'].sum())
groups_sip_dip_sqlsum = pd.DataFrame(dfinit.groupby(['sip','dip'])['sqlnum'].sum())
dfdict = {}
i = 0
for (k1,k2),groupi in groups_sip_dip:
    dfexpi_sip_dip = groupi
    dfexpi_sip_dip['numSum'] = groups_sip_dip_sum['num'][i]
    dfexpi_sip_dip['sqlnumSum'] = groups_sip_dip_sqlsum['sqlnum'][i]
    dfdict[i] = dfexpi_sip_dip
    i+=1
    
dfdict_pd = pd.concat(list(dfdict.values()), ignore_index=True)

df = copy.deepcopy(dfdict_pd)

for i in range(df.shape[0]):
    df.loc[i,'sip'] = socket.ntohl(struct.unpack("I",socket.inet_aton(str(df.loc[i,'sip'])))[0])
    df.loc[i,'dip'] = socket.ntohl(struct.unpack("I",socket.inet_aton(str(df.loc[i,'dip'])))[0])

df['tih'] = pd.to_datetime(df['tih'], format='%Y-%m-%d')
df['hour']=df['tih'].dt.hour
df['No.'] = ""#####Set No. for final splicing
for i in range(df.shape[0]):
    df.loc[i,'No.']=1+i/10000
df.head()


d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,tih,sip,dip,num,difsport,difpw,difinfo,difuser,sqlnum,difsqlinfo,dbtype,numSum,sqlnumSum,hour,No.
0,2019-05-31 00:00:00,171018619,171040675,1,1,1,1,1,0,0,0,93,33,0,1
1,2019-05-31 01:00:00,171018619,171040675,1,1,1,1,1,0,0,0,93,33,1,1.0001
2,2019-05-31 02:00:00,171018619,171040675,1,1,1,1,1,0,0,0,93,33,2,1.0002
3,2019-05-31 03:00:00,171018619,171040675,1,1,1,1,1,0,0,0,93,33,3,1.0003
4,2019-05-31 04:00:00,171018619,171040675,1,1,1,1,1,0,0,0,93,33,4,1.0004


In [5]:
min_max_scaler_01 = preprocessing.MinMaxScaler(feature_range=(1,10), copy=True)

mmfeature=[x for x in df.columns if x not in ['tih', 'No.','dbtype']]

for col in df[mmfeature]:
    df[[col]]=min_max_scaler_01.fit_transform(df[[col]])
df = df.loc[:, (df != 0).any(axis=0)]
df.head()

,tih,sip,dip,num,difsport,difpw,difinfo,difuser,sqlnum,difsqlinfo,dbtype,numSum,sqlnumSum,hour,No.
0,2019-05-31 00:00:00,1.000685,9.957534,1.000702,1.000702,1.000702,10.0,4.0,1.0,1.0,0,1.000364,1.000122,1.000000,1
1,2019-05-31 01:00:00,1.000685,9.957534,1.000702,1.000702,1.000702,10.0,4.0,1.0,1.0,0,1.000364,1.000122,1.391304,1.0001
2,2019-05-31 02:00:00,1.000685,9.957534,1.000702,1.000702,1.000702,10.0,4.0,1.0,1.0,0,1.000364,1.000122,1.782609,1.0002
3,2019-05-31 03:00:00,1.000685,9.957534,1.000702,1.000702,1.000702,10.0,4.0,1.0,1.0,0,1.000364,1.000122,2.173913,1.0003
4,2019-05-31 04:00:00,1.000685,9.957534,1.000702,1.000702,1.000702,10.0,4.0,1.0,1.0,0,1.000364,1.000122,2.565217,1.0004


In [6]:
cakmfeature=[x for x in df.columns if x not in ['tih','dbtype']]
dfea = df
points=dfea[cakmfeature].values
points####Get the array representation of the data points

array([[1.0006849344026705, 9.957534404488001, 1.0007016996725402, ...,
        1.0001224667525446, 1.0, 1.0],
       [1.0006849344026705, 9.957534404488001, 1.0007016996725402, ...,
        1.0001224667525446, 1.391304347826087, 1.0001],
       [1.0006849344026705, 9.957534404488001, 1.0007016996725402, ...,
        1.0001224667525446, 1.7826086956521738, 1.0002],
       ...,
       [9.51135638151709, 10.0, 1.0028067986901605, ...,
        1.0000111333411403, 5.304347826086957, 1.1627],
       [9.51135638151709, 10.0, 1.0021050990176203, ...,
        1.0000111333411403, 4.130434782608695, 1.1628],
       [9.51135638151709, 10.0, 1.0021050990176203, ...,
        1.0000111333411403, 8.043478260869566, 1.1629]], dtype=object)

In [7]:
class Canopy:
    dataset = []

    def __init__(self, dataset, t1, t2):
        self.dataset = dataset
        self.t1 = t1
        self.t2 = t2

    def euclidean_distance(self, point1, point2):
#         return math.sqrt(pow(point1[0]-point2[0], 2) + pow(point1[1]-point2[1], 2))
        Sum=0
        for i in range(len(point1)):
            Sum += pow(point1[i]-point2[i],2)
        return math.sqrt(Sum)

    def get_index(self):
        return np.random.randint(len(self.dataset))

    def find_cluster_by_canopy(self):
        canopy_cluster = []
        while(len(self.dataset) != 0):
            center_set = []
            delete_set = []
            index = self.get_index()
            center_point = self.dataset[index]
            self.dataset = np.delete(self.dataset, index, 0)
            for i in range(len(self.dataset)):
                point = self.dataset[i]
                distance = self.euclidean_distance(point, center_point)
                if distance < self.t1:
                    center_set.append(point)
                if distance < self.t2:
                    delete_set.append(i)
            self.dataset = np.delete(self.dataset, delete_set, 0)
            canopy_cluster.append((center_point, center_set))
            canopy_cluster = [cluster for cluster in canopy_cluster if len(cluster[1]) > 1]
        return canopy_cluster


In [8]:
class KMeans:
    dataset = []
    center_pointset = []

    def __init__(self, dataset, center_pointset, K):
        self.dataset = dataset
        self.center_pointset = center_pointset
        self.K = K

    def euclidean_distance(self, point1, point2):
#         return math.sqrt(pow(point1[0]-point2[0], 2) + pow(point1[1]-point2[1], 2))
        Sum=0
        for i in range(len(point1)):
            Sum += pow(point1[i]-point2[i],2)
        return math.sqrt(Sum)

    def set_euclidean_distance(self, set1, set2):
        if len(set1) == 0 or len(set2) == 0:
            return 1
        flag = 0
        for i in range(len(set1)):
            if self.euclidean_distance(set1[i], set2[i]) != 0:
                flag = 1
                break
        return flag

    def find_center_point(self, list):
        final_list=[]
        length = len(list)
        datalen = df[cakmfeature].shape[1]
        lst = [0]*datalen
        if length != 0:
            for data in list:
                for i in range(datalen):
                    lst[i] += data[i]
            for j in range(datalen):
                final_list.append(lst[j] // length)      
            return final_list
        else:
            print(list)

    def find_cluster_by_kmeans(self):
        kmeans_clusters = []
        count = 0
        old_center_pointset = self.center_pointset
        new_center_pointset = []
        flag = self.set_euclidean_distance(old_center_pointset, new_center_pointset)
        while count < 50 and flag != 0:
            if count != 0:
                old_center_pointset = new_center_pointset
            kmeans_clusters = [[] for _ in range(self.K)]
            for data in self.dataset:
                dist = []
                for i in range(len(old_center_pointset)):
                    distance = self.euclidean_distance(data, old_center_pointset[i])
                    dist.append(distance)
                kmeans_clusters[dist.index(min(dist))].append(data.tolist())
            count += 1
            new_center_pointset = []
            for cluster in kmeans_clusters:
                if len(cluster)!=0:
                    new_center_pointset.append(self.find_center_point(cluster))
            flag = self.set_euclidean_distance(old_center_pointset, new_center_pointset)
        return new_center_pointset, kmeans_clusters
    


## Calculate the distance from the cluster center as the anomaly score

### Test

In [9]:
center_points = []

####Test
canopy = Canopy(points, t1=10, t2=5)

canopy_cluster = canopy.find_cluster_by_canopy()
for i in canopy_cluster:
    center_points.append(i[0].tolist())
K = len(center_points)
print("K:",K)

kmeans = KMeans(points, center_points, K)
center_points, kmeans_cluster = kmeans.find_cluster_by_kmeans()

Nolist=[]
dislist=[]
for i in range(len(center_points)):
    for j in range(len(kmeans_cluster[i])):
        distanceij = kmeans.euclidean_distance(kmeans_cluster[i][j],center_points[i]) ###Calculate the distance of each sample from the cluster center
        Nolist.append(round((kmeans_cluster[i][j][-1]-1)*10000))#######Get the original No.
        dislist.append(distanceij)

NoDisdf = pd.DataFrame(Nolist, columns=['Noindex'])
NoDisdf = pd.concat([NoDisdf, pd.DataFrame(dislist,columns=['Distance'])],axis=1)
NoDisdf.sort_values(by="Noindex" , inplace=True, ascending=True)
NoDisdf.reset_index(drop=True, inplace=True)
NoDisdf.head()


K: 12


,Noindex,Distance
0,0,2.217403
1,1,1.872105
2,2,1.548843
3,3,1.264632
4,4,1.051623


In [10]:
###Splicing to the original data
dfinit['Distance'] = NoDisdf['Distance']
finalsort = dfinit.sort_values(["Distance"],ascending=False)
finalsort.reset_index(drop=True, inplace=True)
finalsort.head()

,tih,sip,dip,num,difsport,difpw,difinfo,difuser,sqlnum,difsqlinfo,dbtype,Distance
0,2019-06-12 16,10.49.178.162,10.49.174.157,18,18,18,1,1,4,2,db,11.346679
1,2019-06-10 09,10.49.231.21,10.49.253.35,71,71,71,1,1,70,2,db,9.001339
2,2019-06-03 11,10.56.92.187,10.24.249.104,5341,5341,5341,1,1,0,0,0,6.935126
3,2019-06-11 11,10.49.137.87,10.49.223.117,139,139,139,1,1,62,2,db,6.461617
4,2019-06-13 10,10.56.92.187,10.24.249.104,6563,6563,6563,1,1,6938,13,db,6.059389


### BatchTest For ROC&PR

In [13]:
#############Defining Parameters and Functions#############
IPnumlist = pd.concat([dfinit["sip"], dfinit["dip"]], axis=0)
IPnumlist=IPnumlist.drop_duplicates()
IPnumlist.reset_index(drop=True, inplace=True)
IPnum= IPnumlist.shape[0]

dfabcorrect=['10.49.223.194','10.49.212.206','10.49.141.206','10.56.34.134','10.49.253.35',
             '10.49.179.13','10.56.148.80',
             '10.32.240.119','10.49.21.125','10.56.148.238']    
def Metric(abIP,TP,FN):
    FP=abIP-TP
    TN=IPnum-abIP-FN
    R= TP/(TP+FN)
    TPR = TP/(TP+FN)
    FPR = FP/(TN+FP)
    P=TP/(TP+FP)
    return round(R, 4),round(TPR, 4),round(FPR, 4),round(P, 4)

##########Take different T1 T2
# T1=10
# T2=1
# T1=5
# T2=1
Maxgas = round(math.sqrt((df[cakmfeature].shape[1]-1) * ((10-1)**2)))######31######This is the maximum possible distance between two points
# Maxgas

#######Define logdf to store logs for visualization
Logdf = pd.DataFrame(columns=('Model','T1','T2','K','ROC_AUC','PRC_AUC','TimeCost'))
# for interval in range(Maxgas): 
for T1 in range(2,Maxgas):
    for T2 in range(1,T1):
    
    
        #######Define Lists to keep TPR FPR R P
        TPRlist = []
        FPRlist = []
        Plist = []
        Rlist = []

        time_start = time.time()

        center_points = []

        canopy = Canopy(points, t1=T1, t2=T2)
#         interval += 5###
        canopy_cluster = canopy.find_cluster_by_canopy()
        for i in canopy_cluster:
            center_points.append(i[0].tolist())
        K = len(center_points)
        # print("K:",K)
        kmeans = KMeans(points, center_points, K)
        center_points, kmeans_cluster = kmeans.find_cluster_by_kmeans()

        Nolist=[]
        dislist=[]
        for i in range(len(center_points)):
            for j in range(len(kmeans_cluster[i])):
                distanceij = kmeans.euclidean_distance(kmeans_cluster[i][j],center_points[i]) ###Calculate the distance of each sample from the cluster center
                Nolist.append(round((kmeans_cluster[i][j][-1]-1)*10000))
                dislist.append(distanceij)

        NoDisdf = pd.DataFrame(Nolist, columns=['Noindex'])
        NoDisdf = pd.concat([NoDisdf, pd.DataFrame(dislist,columns=['Distance'])],axis=1)
        NoDisdf.sort_values(by="Noindex" , inplace=True, ascending=True)
        NoDisdf.reset_index(drop=True, inplace=True)

        dfinit['Distance'] = NoDisdf['Distance']
        finalsort = dfinit.sort_values(["Distance"],ascending=False)
        finalsort.reset_index(drop=True, inplace=True)

        time_end = time.time()
        cost = round((time_end - time_start),4)


        num=1
        while num < finalsort.shape[0]+1:
            ab_finalsort= finalsort[0:num]
            num += 1
            dfabip=pd.concat([ab_finalsort["sip"], ab_finalsort["dip"]], axis=0)
            dfabip=dfabip.drop_duplicates()
            dfabip.reset_index(drop=True, inplace=True)
            dfabip.tolist()
            dfabip_set = set(dfabip)

            dfabcorrect_set = set(dfabcorrect)
            bingo=(dfabip_set)&(dfabcorrect_set)

            abIPi=len(dfabip)
            TPi=len(bingo)
            FNi=len(dfabcorrect)-TPi

            R,TPR,FPR,P = Metric(abIPi,TPi,FNi)
            TPRlist.append(TPR)
            FPRlist.append(FPR)
            Plist.append(P)
            Rlist.append(R)

        ROC = pd.DataFrame(FPRlist, columns=['FPR'])
        ROC = pd.concat([ROC, pd.DataFrame(TPRlist,columns=['TPR'])],axis=1)
        PR = pd.DataFrame(Rlist, columns=['R'])
        PR = pd.concat([PR, pd.DataFrame(Plist,columns=['P'])],axis=1)



        ####Importance drop_duplicates
        ROC=ROC.drop_duplicates()
        PR=PR.drop_duplicates()

        ####Importance sort_values
        ROC.sort_values(by=['FPR','TPR'],inplace=True,ascending=True)
        PR.sort_values(by=['R','P'],inplace=True,ascending=True)
        ROC.reset_index(drop=True, inplace=True)
        PR.reset_index(drop=True, inplace=True)

        #####Importance Add the last line For ROC
        ROC.loc[ROC.shape[0]] = [1,ROC.loc[ROC.shape[0]-1,'TPR']]
        #####Importance Add the first line For PR
        PR.loc[-1] = [0,PR.loc[0,'P']]
        PR.index = PR.index + 1
        PR.sort_index(inplace=True)

        #####AUC
        ROC_AUC = np.trapz(list(ROC['TPR']),list(ROC['FPR'])).round(4)
        PRC_AUC = np.trapz(list(PR['P']),list(PR['R'])).round(4)

        Logdf = Logdf.append({'Model':"canopy_kmeans",'T1':T1,'T2':T2,'K':K,'ROC_AUC':ROC_AUC,'PRC_AUC':PRC_AUC,'TimeCost':cost}, ignore_index=True)

Logdf.to_csv("Results/Canopy_Kmeans_Batch/All_Logs_canopykmeans.csv",index=0)
print(Logdf)

             Model  T1  T2    K  ROC_AUC  PRC_AUC  TimeCost
0    canopy_kmeans   2   1   95    0.254   0.1921    9.1478
1    canopy_kmeans   3   1  117    0.234   0.1851   10.7734
2    canopy_kmeans   3   2   40    0.226   0.1875    2.3473
3    canopy_kmeans   4   1  118    0.226   0.1846   11.6721
4    canopy_kmeans   4   2   40    0.254   0.1921    2.6423
5    canopy_kmeans   4   3   22    0.270   0.1963    1.1625
6    canopy_kmeans   5   1  131    0.210   0.1812   11.2557
7    canopy_kmeans   5   2   45    0.226   0.1875    2.8902
8    canopy_kmeans   5   3   26    0.242   0.1915    0.9794
9    canopy_kmeans   5   4   17    0.258   0.1929    0.4080
10   canopy_kmeans   6   1  127    0.218   0.1825   12.1858
11   canopy_kmeans   6   2   42    0.238   0.1945    2.1746
12   canopy_kmeans   6   3   22    0.270   0.1982    0.5373
13   canopy_kmeans   6   4   17    0.262   0.1934    0.4123
14   canopy_kmeans   6   5   13    0.258   0.1929    0.4232
15   canopy_kmeans   7   1  128    0.218

In [14]:
# Logdf

print(Logdf['ROC_AUC'].mean())
print(Logdf['PRC_AUC'].mean())
print(Logdf['TimeCost'].mean())

0.23128275862068975
0.18987862068965544
1.2957593103448264
